### Include the following code to write sqlite in jupyter notebook
!pip install ipython-sql

# Attribution Queries
CoolTShirts sells shirts of all kinds, as long as they are T-shaped and cool. Recently, CTS started a few marketing campaigns to increase website visits and purchases. Using touch attribution, they’d like to map their customers’ journey: from initial visit to purchase. They can use that information to optimize their marketing campaigns. This project will guide you through some of that process.

### Code to prime the project

In [1]:
import pandas as pd
import sqlite3

%load_ext sql
%config SqlMagic.displaylimit = None
%sql sqlite:///page_visits.db

displaylimit: Value None will be treated as 0 (no limit)

Connecting to 'sqlite:///page_visits.db'

## Get familiar with CoolTShirts

### Task 1
How many campaigns and sources does CoolTShirts use? Which source is used for each campaign?

Use three queries:

* one for the number of distinct campaigns,
* one for the number of distinct sources,
* one to find how they are related.

In [17]:
%%sql
SELECT COUNT(DISTINCT utm_campaign)
FROM page_visits;

Running query in 'sqlite:///page_visits.db'

COUNT(DISTINCT utm_campaign)
8


In [19]:
%%sql
SELECT COUNT(DISTINCT utm_source)
FROM page_visits;

Running query in 'sqlite:///page_visits.db'

COUNT(DISTINCT utm_source)
6


In [21]:
%%sql
SELECT utm_campaign, utm_source
FROM page_visits
GROUP BY utm_campaign;

Running query in 'sqlite:///page_visits.db'

utm_campaign,utm_source
cool-tshirts-search,google
getting-to-know-cool-tshirts,nytimes
interview-with-cool-tshirts-founder,medium
paid-search,google
retargetting-ad,facebook
retargetting-campaign,email
ten-crazy-cool-tshirts-facts,buzzfeed
weekly-newsletter,email


### Task 2
What pages are on the CoolTShirts website?

Find the distinct values of the `page_name` column.

In [28]:
%%sql
SELECT DISTINCT page_name
FROM page_visits;

Running query in 'sqlite:///page_visits.db'

page_name
1 - landing_page
2 - shopping_cart
3 - checkout
4 - purchase


## What is the user journey?

### Task 3
How many first touches is each campaign responsible for?

You’ll need to use a first-touch query. Group by campaign and count the number of first touches for each.

In [33]:
%%sql
WITH first_touch AS (
  SELECT user_id, utm_campaign, MIN(timestamp)
  FROM page_visits
  GROUP BY user_id
)
SELECT ft.utm_campaign, COUNT(*) AS No_first_touches
FROM first_touch AS ft
GROUP BY 1
ORDER BY 2 DESC;

Running query in 'sqlite:///page_visits.db'

utm_campaign,No_first_touches
interview-with-cool-tshirts-founder,590
getting-to-know-cool-tshirts,577
ten-crazy-cool-tshirts-facts,544
cool-tshirts-search,164
weekly-newsletter,42
retargetting-ad,31
retargetting-campaign,17
paid-search,14


### Task 4
How many last touches is each campaign responsible for?

Starting with the last-touch query from the lesson, group by campaign and count the number of last touches for each.

In [42]:
%%sql
WITH last_touch AS (
  SELECT user_id, utm_campaign, MAX(timestamp)
  FROM page_visits
  GROUP BY user_id
)
SELECT lt.utm_campaign, COUNT(*) AS No_last_touches
FROM last_touch AS lt
GROUP BY 1
ORDER BY 2 DESC;

Running query in 'sqlite:///page_visits.db'

utm_campaign,No_last_touches
retargetting-ad,412
weekly-newsletter,405
getting-to-know-cool-tshirts,272
retargetting-campaign,228
ten-crazy-cool-tshirts-facts,226
interview-with-cool-tshirts-founder,207
paid-search,164
cool-tshirts-search,65


### Task 5
How many visitors make a purchase?

Count the distinct users who visited the page named `4 - purchase`.

In [47]:
%%sql
SELECT COUNT(DISTINCT user_id)
FROM page_visits
WHERE page_name = '4 - purchase';

Running query in 'sqlite:///page_visits.db'

COUNT(DISTINCT user_id)
361


### Task 6
How many last touches on the purchase page is each campaign responsible for?

This query will look similar to your last-touch query, but with an additional `WHERE` clause.

In [57]:
%%sql
WITH last_touch AS (
  SELECT user_id, utm_campaign,page_name, MAX(timestamp)
  FROM page_visits
  GROUP BY user_id
)
SELECT lt.utm_campaign, COUNT(DISTINCT lt.user_id) AS No_users_lt_purchase_page
FROM last_touch AS lt
WHERE page_name = '4 - purchase'
GROUP BY 1
ORDER BY 2 DESC;

Running query in 'sqlite:///page_visits.db'

utm_campaign,No_users_lt_purchase_page
weekly-newsletter,99
retargetting-ad,97
retargetting-campaign,48
paid-search,44
ten-crazy-cool-tshirts-facts,8
getting-to-know-cool-tshirts,8
interview-with-cool-tshirts-founder,6
cool-tshirts-search,1


### Task 7
CoolTShirts can re-invest in 5 campaigns. Given your findings in the project, which should they pick and why?

In [59]:
%%sql
WITH last_touch_pg_1 AS (
    SELECT user_id, utm_campaign,page_name, MAX(timestamp) AS last_touch_pg1_at
    FROM page_visits
    WHERE page_name LIKE '1%'
    GROUP BY user_id
),
  last_touch_pg_2 AS (
    SELECT user_id, utm_campaign,page_name, MAX(timestamp) AS last_touch_pg2_at
    FROM page_visits
    WHERE page_name LIKE '2%'
    GROUP BY user_id
),
  last_touch_funnel_12_left AS (
    SELECT lt_pg1.user_id, lt_pg1.utm_campaign, lt_pg1.last_touch_pg1_at,lt_pg2.last_touch_pg2_at
    FROM last_touch_pg_1 AS lt_pg1
    LEFT JOIN last_touch_pg_2 AS lt_pg2
      ON lt_pg1.user_id = lt_pg2.user_id AND
        lt_pg1.utm_campaign = lt_pg2.utm_campaign
),
  last_touch_funnel_12_right AS (
    SELECT lt_pg2.user_id, lt_pg2.utm_campaign, lt_pg1.last_touch_pg1_at,lt_pg2.last_touch_pg2_at
    FROM last_touch_pg_2 AS lt_pg2
    LEFT JOIN last_touch_pg_1 AS lt_pg1
      ON lt_pg2.user_id = lt_pg1.user_id AND
        lt_pg2.utm_campaign = lt_pg1.utm_campaign
),
  last_touch_funnel_12 AS (
    SELECT *
    FROM last_touch_funnel_12_left
    UNION
    SELECT *
    FROM last_touch_funnel_12_right
),
  last_touch_pg_3 AS (
    SELECT user_id, utm_campaign,page_name, MAX(timestamp) AS last_touch_pg3_at
    FROM page_visits
    WHERE page_name LIKE '3%'
    GROUP BY user_id
),
last_touch_funnel_123_left AS (
    SELECT ltf_12.user_id, ltf_12.utm_campaign, ltf_12.last_touch_pg1_at, ltf_12.last_touch_pg2_at,lt_pg3.last_touch_pg3_at
    FROM last_touch_funnel_12 AS ltf_12
    LEFT JOIN last_touch_pg_3 AS lt_pg3
      ON ltf_12.user_id = lt_pg3.user_id AND
        ltf_12.utm_campaign = lt_pg3.utm_campaign
),
  last_touch_funnel_123_right AS (
    SELECT lt_pg3.user_id, lt_pg3.utm_campaign, ltf_12.last_touch_pg1_at, ltf_12.last_touch_pg2_at,lt_pg3.last_touch_pg3_at
    FROM last_touch_pg_3 AS lt_pg3
    LEFT JOIN last_touch_funnel_12 AS ltf_12
      ON lt_pg3.user_id = ltf_12.user_id AND
        lt_pg3.utm_campaign = ltf_12.utm_campaign
),
  last_touch_funnel_123 AS (
    SELECT *
    FROM last_touch_funnel_123_left
    UNION
    SELECT *
    FROM last_touch_funnel_123_right
),
  last_touch_pg_4 AS (
    SELECT user_id, utm_campaign,page_name, MAX(timestamp) AS last_touch_pg4_at
    FROM page_visits
    WHERE page_name LIKE '4%'
    GROUP BY user_id
),
  last_touch_funnel_1234_left AS (
    SELECT ltf_123.user_id, ltf_123.utm_campaign, ltf_123.last_touch_pg1_at, ltf_123.last_touch_pg2_at, ltf_123.last_touch_pg3_at,lt_pg4.last_touch_pg4_at
    FROM last_touch_funnel_123 AS ltf_123
    LEFT JOIN last_touch_pg_4 AS lt_pg4
      ON ltf_123.user_id = lt_pg4.user_id AND
        ltf_123.utm_campaign = lt_pg4.utm_campaign
),
  last_touch_funnel_1234_right AS (
    SELECT lt_pg4.user_id, lt_pg4.utm_campaign, ltf_123.last_touch_pg1_at, ltf_123.last_touch_pg2_at, ltf_123.last_touch_pg3_at,lt_pg4.last_touch_pg4_at
    FROM last_touch_pg_4 AS lt_pg4
    LEFT JOIN last_touch_funnel_123 AS ltf_123
      ON lt_pg4.user_id = ltf_123.user_id AND
        lt_pg4.utm_campaign = ltf_123.utm_campaign
),
  last_touch_funnel_1234 AS (
    SELECT *
    FROM last_touch_funnel_1234_left
    UNION
    SELECT *
    FROM last_touch_funnel_1234_right
)

SELECT utm_campaign, COUNT(last_touch_pg1_at) AS No_lt_visits_pg1, COUNT(last_touch_pg2_at) AS No_lt_visits_pg2, COUNT(last_touch_pg3_at) AS No_lt_visits_pg3, COUNT(last_touch_pg4_at) AS No_lt_visits_pg4
FROM last_touch_funnel_1234
GROUP BY 1
ORDER BY 5 DESC;

Running query in 'sqlite:///page_visits.db'

utm_campaign,No_lt_visits_pg1,No_lt_visits_pg2,No_lt_visits_pg3,No_lt_visits_pg4
weekly-newsletter,0,0,450,115
retargetting-ad,0,0,445,113
retargetting-campaign,0,0,246,54
paid-search,0,0,179,52
ten-crazy-cool-tshirts-facts,576,559,32,9
getting-to-know-cool-tshirts,612,678,41,9
interview-with-cool-tshirts-founder,623,513,31,7
cool-tshirts-search,168,131,7,2


In [61]:
%%sql
WITH last_touch AS (
  SELECT user_id
  FROM page_visits
  WHERE page_name LIKE '4%'
),
  first_touch AS (
    SELECT utm_campaign, user_id
    FROM page_visits
    WHERE page_name LIKE '1%'
)

SELECT ft.utm_campaign, COUNT(lt.user_id) 
FROM last_touch AS lt
LEFT JOIN first_touch AS ft
  ON lt.user_id = ft.user_id
GROUP BY 1;

Running query in 'sqlite:///page_visits.db'

utm_campaign,COUNT(lt.user_id)
cool-tshirts-search,31
getting-to-know-cool-tshirts,106
interview-with-cool-tshirts-founder,119
ten-crazy-cool-tshirts-facts,111


_They should re-invest into the following:_

1. _interview-with-cool-tshirts-founder_
2. _weekly-newsletter_
3. _retargetting-ad_
4. _ten-crazy-cool-tshirts-facts_
5. _getting-to-know-cool-tshirts_

_This decision is based on which campaigns where able to in conjuction, see visitors finishing their purchase after experience a first contact through another/the same campaign and those campaigns who, based on them being the first point of contact, where most likely to see customers purchasing a product._